# Natural Language to Bug Free SQL

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/syntax_error_free_sql.ipynb).

In this example, we will use Guardrails to generate SQL queries from natural language. We will check that the SQL is free of any syntax errors.

## Objective

We want to generate SQL queries from natural language, and check that the SQL is free of any syntax errors.

## Step 0: Setup

In order to run this example, you will need to install the `sqlvalidator` package. You can do so by running the following command:

In [ ]:
!pip install sqlvalidator

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

In this RAIL spec, we:

1. Create an `output` schema that returns a single key-value pair. The key should be 'generated_sql' and the value should be the SQL query generated from the natural language, which is syntactically correct.

In [ ]:
rail_str = """
<rail version="0.1">

<output>
    <string
        name="generated_sql"
        description="Generate SQL for the given natural language instruction."
        format="bug-free-sql"
        on-fail-bug-free-sql="reask" 
    />
</output>


<prompt>

Generate a valid SQL query for the following natural language instruction:

${nl_instruction}

${complete_json_suffix}
</prompt>


</rail>
"""

!!! note

    In order to ensure that the SQL is syntactically correct, we use `bug-free-sql` as the formatter. This is a light-weight formatter that uses the `sqlvalidator` package to check that the SQL is free of any syntax errors. For your use case, you can create a custom SQL validator that connects to your database and checks that the SQL is valid.

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [ ]:
import guardrails as gd

from rich import print

guard = gd.Guard.from_rail_string(rail_str)

We see the prompt that will be sent to the LLM:

In [ ]:
print(guard.base_prompt)

Here, `nl_language` is the natural language instruction and will be provided by the user at runtime.

## Step 3: Wrap the LLM API call with `Guard`

In [ ]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    prompt_params={
        "nl_instruction": "Select the name of the employee who has the highest salary."
    },
    engine="text-davinci-003",
    max_tokens=2048,
    temperature=0,
)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [ ]:
print(validated_response)

In [ ]:
print(guard.state.most_recent_call.tree)